# Bone Scan 변환 코드

안녕하세요 이석현 교수님. 스타랩스 AI연구원 곽찬우 입니다.


사용 방법에 대해 설명드리겠습니다.


폴더 하나씩 하시는 것이 예외의 발생 가능성이 적어 폴더 하나 씩 변환하시는 것을 추천드립니다.



아래 부분은 Library를 import하는 부분입니다. 만약 No module named 'xxx'이라는 오류가 발생하신다면 셀을 하나 더 만드시고(상단 메뉴의 +버튼) !pip install xxx이라고 입력해 주신 후 실행을 눌러 주시면 됩니다.(상단 메뉴의 재생버튼)


만약 PIL이라는 모듈이 없다고 할 경우 !pip install pillow라고 입력해 주신 후 실행해주시면 됩니다.



In [64]:
import glob
import os
import pydicom as dcm 
import png
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import PIL.ImageOps 
from PIL import Image
import cv2
import PIL
import os
import natsort
import shutil

path = 'C:/Users/BrainK/Downloads/Bone_Scan_TEST/' #이 부분이 폴더 경로 부분 입니다. 폴더 경로를 입력해주시면 됩니다. 만약 \가 입력되었을경우 /로 바꿔주시길 바랍니다.

#현재 dicom파일이 확장자가 없어서 *.*의 형태로 불러옵니다.
#폴더안에 dicom파일 이외의 파일이 있을 경우 오류가 생길 수 있습니다. 
form = '/*/*' # 여러 폴더 다중 처리하실 경우 이 부분을 '/*/*'로 바꿔주세요. /*은 전체 폴더를, *은 모든 파일명을 뜻합니다. 확장자를 제한하고 싶을실 경우 *.dcm의 형태로 제한해주시면 됩니다.

#불러올 dicom파일 목록입니다.
dcms = glob.glob(path+form)

dicompath = path #dicom이 들어있는 폴더의 상의 폴더로써 변환된 이미지가 저장될 폴더입니다.

#ANT, POST별로 폴더를 생성하여 변환합니다.
ant = path + '/ANT'
post = path + '/POST'

#해당 경로에 폴더가있는지 확인하고 없을경우 생성합니다.
if not(os.path.isdir(dicompath)): 
    os.makedirs(os.path.join(dicompath))
    
if not(os.path.isdir(ant)):
    os.makedirs(os.path.join(ant))
    
if not(os.path.isdir(post)):
    os.makedirs(os.path.join(post))

#읽은 dicom파일 목록을 출력해봅니다. 제대로 불러와졌으면 목록이 뜨게 됩니다. []으로 뜨게 되면 아무것도 읽지 못했다는 것으로 경로를 다시 확인해주시기 바랍니다.   
print(dcms)

['C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000316', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000318', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000320', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000322', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000324', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000326', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000328', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000330', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000332', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000336', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000340', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000345', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000347', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000349', 'C:/Users/BrainK/Downloads/Bone_Scan_TEST\\2019_3rd\\00000351', 'C:/Users/BrainK/Downloads/Bone_Scan_TE

### 이전에 교수님께서 제안해주신 방법을 적용하여 변환하는 부분입니다.


### 이전에 설명해드렸던 부분이라 익숙하실 것 같습니다.

In [69]:
for i in range(len(dcms)):
    #print(dcms[i])
    for j in range(0,2):
        tmp_name = os.path.basename(dcms[i]) 
        
        folder_name = dcms[i].replace(path,'')
        folder_name = folder_name.replace('\\' + os.path.basename(dcms[i]),'')
        folder_name = folder_name.replace('\\','')
        
        if(1):
            
            dicom = dcm.dcmread(dcms[i])
            #print(dcms[i])
            #print(dicom)
            dicom_temp = dcm.dcmread(dcms[i-1])
            
            if(len(dicom.pixel_array) == 2 and (dicom.pixel_array.shape[1] > 1000 or (dicom.pixel_array.shape[0] > 1000))):
                shape = dicom.pixel_array[j].shape

                countsAccumulated = dicom.CountsAccumulated #이 변수가 Counts Accumulated값입니다.
                image_2d = dicom.pixel_array[j].astype(float)
                
                ############ 파일명에 ANT, POST 구분이 없어 이전에 있었던 ANT, POST 규칙은 제거하였습니다 ###########
                
                if(j==0):
                    status = '_ANT'
                    if(1):
                        filename_no_dir = 'ANT/' + '[' + dicom.StudyDate + ']_{' + dicom.PatientID + '}_' + str(i) + str(j)  
                    #else:
                    #    filename_no_dir = 'ANT/' + dicom.AccessionNumber
                else:
                    status = '_POST'
                    if(1):
                        filename_no_dir = 'POST/' + '[' + dicom.StudyDate + ']_{' + dicom.PatientID + '}_' + str(i) + str(j)  
                    #else:
                    #    filename_no_dir = 'POST/' + dicom.AccessionNumber
                    
            elif((dicom.pixel_array.shape[1] > 1000) or (dicom.pixel_array.shape[0] > 1000)):
                j=2
                shape = dicom.pixel_array.shape
                countsAccumulated = dicom.CountsAccumulated #이 변수가 Counts Accumulated값입니다.
                image_2d = dicom.pixel_array.astype(float)
                #파일의 이름은 [study date]_{patient ID}의 규칙으로 생성되도록 설정하였습니다.
                if(dcms[i].find('ANT') != -1):
                    status = '_ANT'
                    if(len(dicom.AccessionNumber) < 3):
                        filename_no_dir = 'ANT/' + dicom.PatientID + dicom.StudyDate + '_' + i + j
                    else:
                        filename_no_dir = 'ANT/' + dicom.AccessionNumber + '_' + i + j
                elif((dcms[i].find('POST') != -1) and len(dicom_temp.pixel_array) == 1):
                    status = '_POST'
                    if(len(dicom.AccessionNumber) < 3):
                        filename_no_dir = 'POST/' + dicom.PatientID + dicom.StudyDate + '_' + i + j
                    else:
                        filename_no_dir = 'POST/' + dicom.AccessionNumber + '_' + i + j
                
            w_level = countsAccumulated/100000     #Level을 변경하고 싶으실땐 이 변수를 수정해주시면 되고
            w_width = countsAccumulated/100000 * 2 #width를 변경하고 싶으실떈 이 변수를 수정해주시면 됩니다.

            minw = int(w_level)-(int(w_width))
            maxw = int(w_level)+(int(w_width))

            image_2d[image_2d<minw] = minw 
            image_2d[image_2d>maxw] = maxw

            image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
            image_2d_scaled = np.uint8(image_2d_scaled)

            
            #filename_no_dir = dicom.AccessionNumber
            destination = dicompath + '/' + filename_no_dir + status + '.png'
            destination = destination.replace('.dcm','_')
            #print(destination)
            with open(destination, 'wb') as png_file:
                w = png.Writer(shape[1], shape[0], greyscale=True)
                w.write(png_file, image_2d_scaled)

            #만약 이미지를 반전시킬 필요가 없으면 밑에 세줄을 지워주세요.
            image = Image.open(destination)               
            inverted_image = PIL.ImageOps.invert(image)   
            inverted_image.save(destination)      
        
    #처리된 파일이 출력됩니다.
    
        #print(i)

### 아래부분은 ANT와 POST의 형태로 변환된 이미지를 하나로 합치는 부분입니다.

In [84]:
#이 부분은 cv2의 이미지를 읽는 함수가 한글을 인식하지 못해 다시 정의해 주는 부분입니다.
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8): 
    try: 
        n = np.fromfile(filename, dtype) 
        img = cv2.imdecode(n, flags) 
        img = cv2.cvtColor(img, flags)
        return img 

    except Exception as e: 
        print(e) 
        return None
        
    
def imwrite(filename, img, params=None): 
    try: 
        ext = os.path.splitext(filename)[1] 
        result, n = cv2.imencode(ext, img, params) 
        if result: 
            with open(filename, mode='w+b') as f: 
                n.tofile(f) 
                return True 
        else:
            return False 
    except Exception as e: 
        print(e) 
        return False

In [85]:
done_folder = '/done' #합쳐진 이미지가 저장될 폴더이름입니다.
done_dir = path + done_folder

#해당 폴더가 없으면 생성합니다.
if not(os.path.isdir(done_dir)):
    os.makedirs(os.path.join(done_dir))

In [89]:
#ANT파일과 POST파일을 불러옵니다.
ant_dir = glob.glob(path + '/ANT/*.*')
post_dir = glob.glob(path + '/POST/*.*')

In [88]:
ant_dir

['C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190701]_{170293173}_00_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190701]_{910476443}_10_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{040056793}_50_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{040309243}_60_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{060019023}_30_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{090485523}_100_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{140207853}_20_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{190122683}_40_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{190173923}_90_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{930449713}_80_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190702]_{990328273}_70_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//ANT\\[20190703]_{030

In [90]:
#ANT파일을 기준으로 이미지를 읽습니다.
#만약 파일 변환에 문제가 있어 ANT파일과 POST파일의 목록이 다를경우 오류를 방지합니다.
for i in ant_dir:
    #ANT이미지를 불러옵니다.
    ant_img = cv2.imread(i)
    #ANT이미지에서 POST로 이름을 바꿔 POST도 불러옵니다.
    post_img = cv2.imread(i.replace('ANT','POST'))
    
    #두 이미지를 합칩니다.
    done_img = cv2.hconcat([ant_img,post_img])
    
    #합친 이미지를 저장합니다.
    cv2.imwrite(path + done_folder + '/' + os.path.basename(i),done_img)
    

### 아래 부분은 보내주신 Excel파일과 연동하여 group을 분류하는 부분입니다

In [93]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
#엑셀을 불러옵니다.
patient_data = load_workbook('C:/Users/BrainK/Downloads/Bone Scan (1)/Bone scan (1).xlsx')
#지정한 시트를 불러옵니다.
excel_data = patient_data['Sheet1']
#데이터프레임으로 변환합니다.
df = pd.DataFrame(excel_data.values)

#변환된 데이터 프레임을 출력합니다.
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,ExamDate,Section,Modality,ExamCode,ExamName,PatID,Sex,Age,Birthday,ReportDate,Conclusion,Group,Result,Finding,Recommend,Radiologist,Status,None
1,2019-07-01 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),170293173,F,65,1956-12-05 00:00:00,2019-07-02 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
2,2019-07-01 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),910476443,F,74,1947-12-25 00:00:00,2019-07-02 00:00:00,RCC,2,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
3,2019-07-02 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),990328273,F,52,1969-10-03 00:00:00,2019-07-03 00:00:00,Breast cancer,1,0,No definite evidence of bone metastasis\n,None,이석현(181356)/,C,None
4,2019-07-02 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),040309243,F,65,1957-02-15 00:00:00,2019-07-03 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6504,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),170407993,F,83,1939-04-01 00:00:00,2022-07-01 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
6505,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),220315273,M,67,1955-05-01 00:00:00,2022-07-01 00:00:00,Prostate cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
6506,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),120168183,M,79,1942-10-14 00:00:00,2022-07-01 00:00:00,Bladder tumor,2,0,No definite evidence of bone metastasis\n,None,이석현(181356)/,C,None
6507,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),220334833,F,38,1984-01-06 00:00:00,2022-07-01 00:00:00,Pain,3,0,No active bone lesion\n,None,이석현(181356)/,C,None


In [94]:
#위에서 보시면 column명을 0번째 행으로 설정한 후 0번째 행은 제거해줍니다. 
df.columns = df.iloc[0]
df = df.drop(0)
df['ExamDate'] = df['ExamDate'].astype(str) #datetime(시간형식)형식으로 되어있어서 str형식(일반문자)으로 변환해줍니다.

In [95]:
#파일명의 Study Date와 비교하기위해 동일한 형식으로 변환(제거)합니다.
df['ExamDate'] = df['ExamDate'].replace('-','',regex = True)
df['ExamDate'] = df['ExamDate'].replace('00:00:00','',regex = True)

# 참고하실 내용

여기보시면 제가 result에 9라고 된 환자들을 출력하고 있습니다.

Excel표에 result 부분이 N/A로 표시된 환자들을 임의로 9로 변환, no_final_diagnosis로 표시된 환자들을 8로 변환하였습니다.

위 환자들은 결과에서 제외됩니다.

In [96]:
####
df['Result'][df['Result'] == 9]

39      9
154     9
209     9
250     9
276     9
       ..
6160    9
6203    9
6212    9
6283    9
6327    9
Name: Result, Length: 123, dtype: object

In [109]:
##################################
#result 9 = N/A
#result 8 = no final diagonosis
##################################

#변환이 완료된 파일의 목록을 읽어옵니다.
done_file = done_dir + '/*.png'  
done_file = glob.glob(done_file)
#정렬합니다.
done_file = natsort.natsorted(done_file)

#파일명의 study date 시작 인덱스입니다. [xxxx.xx.xx]규칙입니다. 0번째는 [이니 1번째 인덱스입니다. 모든 파일이 공통이라 1번째 인덱스로 고정해줍니다.
date_start = 1
#인덱싱에 사용할 count 변수입니다.
count = 1
not_count = 0
error_count = 0 
na_count = 0
final_count = 0

error_list = list() # 일치하는 엑셀 정보 없음
not_list = list() # 그룹조건에 없음
for i in done_file:
    #각 파일에서 규칙을 기준으로 인덱스를 조회합니다.
    #모든 파일이 동일한 규칙을 가지고 있기에 동일한 인덱스지만 중간에 파일명 형식이 다른 경우가 있어 각 파일별로 불러옵니다.
    date_end = os.path.basename(i).find(']')
    id_start = os.path.basename(i).find('{') + 1
    id_end = os.path.basename(i).find('}')
    
    #불러온 인덱스로 study date와 patient id를 불러옵니다.
    file_date = os.path.basename(i)[date_start:date_end]
    #비교를 위해 int(정수형)으로 변환하였습니다.
    file_id = int(os.path.basename(i)[id_start:id_end])
    exel_id = df.loc[count]['PatID']
    
    #읽어온 patient id와 study date를 출력합니다.
    #print(file_id, file_date)
    #비교를 위해 변환하여 비교하고 파일의 patient id와 excel의 patient id와 동일한 인덱스를 찾아 temp에 저장합니다. 
    temp = df[(df['PatID'].astype(int) == int(file_id))]
    #patient id에서 일치하는 경우 study date를 한 번 더 비교하여 일치하는 정보를 찾습니다. 
    match = temp[temp['ExamDate'].astype(int) == int(file_date)]
    if(len(match) == 0):
        error_count = error_count + 1 
        error_list.append(i)
        continue
    #위에서 찾은 정보로 해당 파일의 group을 저장합니다.
    group = match['Group'].astype(int)
    
    #데이터프레임에서 원소만 저장합니다.
    group = group.iloc[0]
    
    #비교를 위해 정수형으로 변환하였습니다.
    result = match['Result'].astype(int)
    #데이터프레임에서 원소만 저장합니다.
    result = result.iloc[0]
    
    #group에 따라 지정할 파일 경로입니다.
    group1_Negative = os.path.dirname(i.replace('done','class/Group1/Negative'))
    group1_Positive = os.path.dirname(i.replace('done','class/Group1/Positive'))
    
    group2_Negative = os.path.dirname(i.replace('done','class/Group2/Negative'))
    group2_Positive = os.path.dirname(i.replace('done','class/Group2/Positive'))
    
    group3_Negative = os.path.dirname(i.replace('done','class/Group3/Negative'))
    group3_Positive = os.path.dirname(i.replace('done','class/Group3/Positive'))
    
    #경로가 없을 경우 생성합니다.
    if not(os.path.isdir(group1_Negative)):
        os.makedirs(os.path.join(group1_Negative))
        os.makedirs(os.path.join(group1_Positive))
        
        os.makedirs(os.path.join(group2_Negative))
        os.makedirs(os.path.join(group2_Positive))
        
        os.makedirs(os.path.join(group3_Negative))
        os.makedirs(os.path.join(group3_Positive))
        
    #설정한 group과 조건에 따라 분류합니다.
    #N/A와 no final diagnosis의 경우 제외합니다.
    if(group == 1):
        if(result == 0):
            #print('group1_음성')
            shutil.copy2(i,group1_Negative)
        elif(result == 1):
            #print('group1_음성')
            shutil.copy2(i,group1_Negative)
        elif(result == 2):
            #print('group1_양성')
            shutil.copy2(i,group1_Positive)
        elif(result == 3):
            #print('group1_양성')
            shutil.copy2(i,group1_Positive)
        elif(result == 9):
            print('group1_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group1_no_final_diagnosis')
            final_count = final_count + 1
    elif(group == 2):
        if(result == 0):
            #print('group2_음성')
            shutil.copy2(i,group2_Negative)
        elif(result == 1):
            #print('group2_음성')
            shutil.copy2(i,group2_Negative)
        elif(result == 2):
            #print('group2_양성')
            shutil.copy2(i,group2_Positive)
        elif(result == 3):
            #print('group2_양성')
            shutil.copy2(i,group2_Positive)
        elif(result == 9):
            print('group2_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group2_no_final_diagnosis')
            final_count = final_count + 1
    elif(group == 3):
        if(result == 0):
            #print('group3_음성')
            shutil.copy2(i,group3_Negative)
        elif(result == 1):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 2):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 3):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 9):
            print('group3_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group3_no_final_diagnosis')
            final_count = final_count + 1
    else:
        print('except!_' + i)
        not_count = not_count + 1
        not_list.append(i)
        continue
        
    count = count + 1
print(not_count)
print(error_count)
print(na_count)
print(final_count)

except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190708]_{190174363}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190724]_{100050833}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190801]_{190220723}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190807]_{190041333}_ANT.png
group2_no_final_diagnosis
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190812]_{190224243}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190822]_{000001743}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190826]_{000269223}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190910]_{190268583}_ANT.png
group2_no_final_diagnosis
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20190920]_{950313443}_ANT.png
group2_no_final_diagnosis
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[20191102]_{190174363}_ANT.png
except!_C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\[201911

In [113]:
print(error_list)
display(not_list)

['C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20220615]_{190211003}_ANT.png']


['C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190708]_{190174363}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190724]_{100050833}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190801]_{190220723}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190807]_{190041333}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190812]_{190224243}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190822]_{000001743}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190826]_{000269223}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190910]_{190268583}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20190920]_{950313443}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20191102]_{190174363}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20191106]_{170193153}_ANT.png',
 'C:/Users/BrainK/Downloads/Bone_Scan_TEST//done\\[20191108]_{190285613}_ANT.png',
 'C:

In [71]:
all_list = glob.glob('C:/Users/BrainK/Downloads/Bone_Scan_TEST/ANT/*.png')

In [77]:
date_start = 1
png_list = list()
for i in all_list:
    date_end = os.path.basename(i).find(']')
    id_start = os.path.basename(i).find('{') + 1
    id_end = os.path.basename(i).find('}')
    
    file_date = os.path.basename(i)[date_start:date_end]
    file_id = os.path.basename(i)[id_start:id_end]
    
    png_list.append(file_date + '_' + file_id)
    

In [82]:
count = 0
for i in range(len(png_list)-1):
    if png_list[i] == png_list[i-1]:
        print(png_list[i])
        count = count + 1
        
print(count)

20190704_190191823
20190708_190174363
20190715_150107823
20190725_010366813
20190725_940264373
20190729_950360043
20190801_190071943
20190803_150285093
20190810_910522843
20190817_170435743
20190821_090202903
20190823_190250563
20190829_030228353
20190830_190255603
20190905_960276053
20190917_100309443
20190917_150286923
20190919_190129143
20190919_960229503
20190920_950313443
20190923_030206033
20191116_190061333
20191126_180027213
20200404_190290013
20200404_200096953
20200407_130062213
20200411_130181363
20200413_190343523
20200414_050393113
20200417_140442533
20200417_150294173
20200422_910259843
20200423_200115543
20200425_080346823
20200504_020375203
20200504_200121293
20200504_950328783
20200508_120029793
20200514_110412853
20200515_110002913
20200521_190158733
20200521_930141033
20200522_180135723
20200525_200146713
20200527_960036063
20210531_960330523
20220328_220208013
20220328_220208013
20220328_220208013
20220328_220208013
20220328_220208013
20220328_220208013
20220328_220

In [112]:
class_list = glob.glob('C:/Users/BrainK/Downloads/Bone_Scan_TEST/class/*/*/*.png')
print(len(class_list))

6301
